**Cloning YOLOv5 Repository**

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5

**Installing Dependencies and Setting Up YOLOv5**

In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))



**Loading API Key and Downloading Dataset from Roboflow**

In [ ]:
#loading the API key for the dataset
!pip install -q roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="//paste your api key or get your unique syntax for your dataset") #change your api key here
project = rf.workspace("image-segmentation-lfxjl").project("specs-vzzww")
version = project.version(1)
dataset = version.download("yolov5")

**Defining Number of Classes Based on YAML Configuration**

In [ ]:
# define number of classes based on YAML
import yaml
with open("//PATH TO YOUR DATA YAML FILE", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

#or you can run these commands  before this block for easy access to path
   #%pwd
   #dataset.location
   #%cd /content/yolov5
   #dataset.location
   #%cat {dataset.location}/data.yaml
num_classes

In [ ]:
#this is the model configuration we will using for this project
%cat /content/yolov5/models/yolov5s.yaml

**Customizing iPython Writefile Functionality for Variable Writing**

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

**Writing Custom YOLOv5s Configuration YAML File**

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

anchors:
  - [382,166]  # Simplified for P3/8 (assuming this matches the resized object size closely)
  - [382,166]  # Simplified for P4/16
  - [382,166]  # Simplified for P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

**Training YOLOv5s on Custom Data for 200 Epochs**

In [ ]:
# train yolov5s on custom data for 200 epochs
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 200 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights 'yolov5s.pt' --name yolov5s_results  --cache


**Interpreting Training Results for YOLOv5s**

In [ ]:
#interpreture the results
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/yolov5s_results/results.png', width=1000)  # view results.png

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5s_results/train_batch0.jpg', width=900)

In [ ]:
# trained weights are saved by default in our weights folder
#%ls runs/train/yolov5s_results/weights